# Bayesian analysis of emotion-mediated memory as a function of power and depression

In [ ]:
import arviz as az
import bambi as bmb
import argparse
from os.path import join
import pandas as pd
from pandas.api.types import CategoricalDtype
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from Bayesian_model_utils import run_model, plot_res, print_latex_table, plot_predictions

In [ ]:
# Model 1: Effect of depression and power and arousal on memory and binned valence

BeckScores = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/Beck_Scores.csv')
BeckScores.rename(columns={'Subject Code':'subj'}, inplace=True)
BeckScores = BeckScores.drop(columns=['Unnamed: 0'])

pow_df_depr = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/final_NHB_power_data_revised.csv', memory_map=True)
pow_df_depr = pow_df_depr.merge(BeckScores, on='subj')


# rename the electrodes to properly do the elec random effects 
pow_df_depr.elec = pow_df_depr.apply(lambda x: f'{x.subj}_{x.elec}', axis=1)

pow_df_depr['valence'] = pow_df_depr['valence'] - 0.5

# Set categorical orders 

cat_type = CategoricalDtype(categories=['Amy', 'Hipp'], ordered=True)
pow_df_depr.region = pow_df_depr.region.astype(cat_type)
cat_type = CategoricalDtype(categories=['right', 'left'], ordered=True)
pow_df_depr.hemi = pow_df_depr.hemi.astype(cat_type)

for band in ['hfa', 'theta']: # theta 
            
    model_df = pow_df_depr[pow_df_depr.band==band]

    y = 'memory'
    X = ['arousal', 'valence', 'region', 'hemi', 'power', 'BDI'] 
    
    Intx = ['power:arousal', 
            'power:valence',
            'power:hemi',
            'power:region',
            'power:BDI',
            'power:BDI:arousal',
            'power:BDI:valence']

    rand_effect = ['subj', 'elec']
    categorical = ['hemi', 'region']

    # Drop nan data
    model_df = model_df.dropna(subset=X)
    label = (f"{y}" + "_{}"*len(X)).format(*X) + f'_{band}' + '_elec_rand_effect'+ 'continuousBDI' + '_nosoz'

    run_model(model_df, y, X, Intx, rand_effect, rand_slopes=False, categorical=categorical,
             chains=4, cores=4, tune=1000, draws=1000, label=label)

            

In [ ]:
# save out the HDI plot of all the fixed effects
band='gamma'
output_dir = '/home1/salman.qasim/Salman_Project/FR_Emotion/BayesModels'
for region in ['Hipp', 'Amy']: 
    for depr in ['low', 'high']: 
        y = 'memory'
        X = ['arousal', 'CV', 'power'] 
        label = (f"{y}" + "_{}"*len(X)).format(*X) + region + band + f'{depr}_BDI'
        results = az.from_netcdf(f'{output_dir}/{label}_model')
        axes = az.plot_forest(results,
                               kind='ridgeplot',
                               var_names=[f'^{x}' for x in X],
                               filter_vars="regex",
                               colors='lightgray',
                               combined=True,
                               hdi_prob=0.95,
                               figsize=(9, 7))
        plt.vlines(0, plt.ylim()[0], plt.ylim()[1], color = 'black')
        plt.xlim([-0.4, 0.4])
        plt.savefig(join(output_dir, f'{label}_HDIplot.pdf'), dpi=300)
        plt.close()